In [2]:
import pandas as pd
import numpy as np

In [3]:
# !pip install pandas

## Loading the groundtruth data

In [9]:
data = pd.read_csv(r"E:\Work\Data_Science\Projects\Custom_NER_Spacy3.5\data\ner.csv", encoding='cp1252', on_bad_lines='warn')
data.head()

Skipping line 281837: expected 25 fields, saw 34



,Unnamed: 0,lemma,next-lemma,next-next-lemma,next-next-pos,next-next-shape,next-next-word,next-pos,next-shape,next-word,...,prev-prev-lemma,prev-prev-pos,prev-prev-shape,prev-prev-word,prev-shape,prev-word,sentence_idx,shape,word,tag
0,0,thousand,of,demonstr,NNS,lowercase,demonstrators,IN,lowercase,of,...,__start2__,__START2__,wildcard,__START2__,wildcard,__START1__,1.0,capitalized,Thousands,O
1,1,of,demonstr,have,VBP,lowercase,have,NNS,lowercase,demonstrators,...,__start1__,__START1__,wildcard,__START1__,capitalized,Thousands,1.0,lowercase,of,O
2,2,demonstr,have,march,VBN,lowercase,marched,VBP,lowercase,have,...,thousand,NNS,capitalized,Thousands,lowercase,of,1.0,lowercase,demonstrators,O
3,3,have,march,through,IN,lowercase,through,VBN,lowercase,marched,...,of,IN,lowercase,of,lowercase,demonstrators,1.0,lowercase,have,O
4,4,march,through,london,NNP,capitalized,London,IN,lowercase,through,...,demonstr,NNS,lowercase,demonstrators,lowercase,have,1.0,lowercase,marched,O


In [10]:
data.sentence_idx.value_counts()

2549.0     140
11994.0    132
608.0      124
5805.0     122
6344.0     120
          ... 
37093.0      2
8412.0       2
39874.0      2
40249.0      2
38917.0      1
Name: sentence_idx, Length: 35177, dtype: int64

In [11]:
sentences = data.sentence_idx.value_counts().index[:10000]
#sentences = sentences.index
#sentences

In [7]:
len(sentences)

10000

In [12]:
new_data = data[data['sentence_idx'].isin(sentences)]
new_data.head()

,Unnamed: 0,lemma,next-lemma,next-next-lemma,next-next-pos,next-next-shape,next-next-word,next-pos,next-shape,next-word,...,prev-prev-lemma,prev-prev-pos,prev-prev-shape,prev-prev-word,prev-shape,prev-word,sentence_idx,shape,word,tag
0,0,thousand,of,demonstr,NNS,lowercase,demonstrators,IN,lowercase,of,...,__start2__,__START2__,wildcard,__START2__,wildcard,__START1__,1.0,capitalized,Thousands,O
1,1,of,demonstr,have,VBP,lowercase,have,NNS,lowercase,demonstrators,...,__start1__,__START1__,wildcard,__START1__,capitalized,Thousands,1.0,lowercase,of,O
2,2,demonstr,have,march,VBN,lowercase,marched,VBP,lowercase,have,...,thousand,NNS,capitalized,Thousands,lowercase,of,1.0,lowercase,demonstrators,O
3,3,have,march,through,IN,lowercase,through,VBN,lowercase,marched,...,of,IN,lowercase,of,lowercase,demonstrators,1.0,lowercase,have,O
4,4,march,through,london,NNP,capitalized,London,IN,lowercase,through,...,demonstr,NNS,lowercase,demonstrators,lowercase,have,1.0,lowercase,marched,O


In [13]:
new_data.tag.value_counts()

O        424516
B-geo     17934
B-org      9841
B-tim      9453
I-per      8744
I-org      8570
B-per      8208
B-gpe      8075
I-geo      3876
I-tim      3057
B-art       255
B-eve       207
I-eve       168
I-art       156
I-gpe       136
B-nat       124
I-nat        46
Name: tag, dtype: int64

In [14]:
# # parsing data 
# data_json = []
# for i in sentences:
#     s = ""
#     entities = []
#     sub_data = new_data[new_data['sentence_idx']==i]
#     beg = 0
#     for j in sub_data.index:
#         word = sub_data.loc[j,'word']
#         s = s+' '+word
#         tag = sub_data.loc[j,'tag']
#         if tag != 'O':
#             start = beg+s[beg:].find(word)
#             end = start+len(word)
#             beg=end
#             entities.append([start, end, tag])
#     data_json.append([s, {'entities':entities}])
# #print(data_json)

    

### Parsing the data to follow this convention:-
 ["this is our text", {'entities':[[start, end, label], [start, end, label]]},</br>
"this is our text", {'entities':[[start, end, label]]}...]

In [15]:
# parsing data 
data_json = []
for i in sentences:
    s = ""
    beg = 0
    entities = []
    sub_data = new_data[new_data['sentence_idx']==i]
    for j in sub_data.index:
        word = sub_data.loc[j,'word']
        s = s+word+' '
        start = beg
        end = start+len(word)
        beg = end+1
        tag = sub_data.loc[j,'tag']
        if tag != 'O':
            entities.append([start, end, tag])
    data_json.append([s, {'entities':entities}])
#print(data_json)

    

In [16]:
' Last week , Mr. Annan said the U.N. force will have to be bigger and better equipped , which , he said , will require the participation of big and powerful countries with highly trained troops - including the U.S. U.S. officials have said they hope to arrange a U.N. mission to Darfur by the end of this month . Last week , Mr. Annan said the U.N. force will have to be bigger and better equipped , which , he said , will require the participation of big and powerful countries with highly trained troops - including the U.S. U.S. officials have said they hope to arrange a U.N. mission to Darfur by the end of this month .'[17:22]

'Annan'

In [17]:
len(data_json)

10000

In [18]:
#data_json

In [19]:
# train dev test split
import random
test_list = random.sample(range(0, 10000), 1500)
test_data_json = [ele for i,ele in enumerate(data_json) if i in test_list]
train_data_json = [ele for i,ele in enumerate(data_json) if i not in test_list]
dev_list = random.sample(range(0, 8500), 1500)
dev_data_json = [ele for i,ele in enumerate(train_data_json) if i in dev_list]
train_data_json = [ele for i,ele in enumerate(train_data_json) if i not in dev_list]
len(test_data_json), len(dev_data_json), len(train_data_json)

(1500, 1500, 7000)

In [21]:
import json
test_data_json = json.dumps(test_data_json)
dev_data_json = json.dumps(dev_data_json)
train_data_json = json.dumps(train_data_json)
data_json = json.dumps(data_json)

In [22]:
# save in json format
with open(r'E:\Work\Data_Science\Projects\Custom_NER_Spacy3.5\data\Train\all\data_json.json', 'w') as f:
    f.write(data_json)

with open(r'E:\Work\Data_Science\Projects\Custom_NER_Spacy3.5\data\Train\train\train_data_json.json', 'w') as f:
    f.write(train_data_json)

with open(r'E:\Work\Data_Science\Projects\Custom_NER_Spacy3.5\data\Train\dev\dev_data_json.json', 'w') as f:
    f.write(dev_data_json)
    
with open(r'E:\Work\Data_Science\Projects\Custom_NER_Spacy3.5\data\Train\test\test_data_json.json', 'w') as f:
    f.write(test_data_json)

In [24]:
# read the json back
with open(r'E:\Work\Data_Science\Projects\Custom_NER_Spacy3.5\data\Train\train\train_data_json.json', 'r') as f:
    train_data_json = json.load(f)

with open(r'E:\Work\Data_Science\Projects\Custom_NER_Spacy3.5\data\Train\dev\dev_data_json.json', 'r') as f:
    dev_data_json = json.load(f)
    
with open(r'E:\Work\Data_Science\Projects\Custom_NER_Spacy3.5\data\Train\test\test_data_json.json', 'r') as f:
    test_data_json = json.load(f)
    

## Function to convert json to spacy format for spacy NER 

In [77]:
# convert to spacy format of train and dev, save them
import spacy
from spacy.tokens import DocBin
def json2spacy_ner(ip, op_path):
    nlp = spacy.blank('en')
    db = DocBin()
    c = 0
    for text, annotation in ip:
        c+=1
        doc = nlp(text)
        ents = []
        for start, end, label in annotation['entities']:
            span = doc.char_span(start, end, label=label)
            ents.append(span)
        doc.ents = ents
        db.add(doc)
    db.to_disk(op_path)
    
json2spacy_ner(train_data_json, r"E:\Work\Data_Science\Projects\Custom_NER_Spacy3.5\data\Train_ner\train\train.spacy")
json2spacy_ner(dev_data_json, r"E:\Work\Data_Science\Projects\Custom_NER_Spacy3.5\data\Train_ner\dev\dev.spacy")

## Function to convert spacy ner format to spacy Span Categorizer format

In [2]:
from pathlib import Path
import typer
import spacy
from spacy.tokens import DocBin


def json2spacy_spancat(src, dest):
    """
    Set the NER data into the doc.spans, under a given key.
    The SpanCategorizer component uses the doc.spans, so that it can work with
    overlapping or nested annotations, which can't be represented on the
    per-token level.
    """
    nlp = spacy.blank('en')
    span_key = 'sc'
    docbin = DocBin().from_disk(src)
    docs = list(docbin.get_docs(nlp.vocab))
    for doc in docs:
        doc.spans[span_key] = list(doc.ents)
    DocBin(docs=docs).to_disk(dest)

json2spacy_spancat(r"E:\Work\Data_Science\Projects\Custom_NER_Spacy3.5\data\Train\train\train.spacy", 
                   r"E:\Work\Data_Science\Projects\Custom_NER_Spacy3.5\data\Train_spancat\train\train.spacy")
json2spacy_spancat(r"E:\Work\Data_Science\Projects\Custom_NER_Spacy3.5\data\Train\dev\dev.spacy", 
                   r"E:\Work\Data_Science\Projects\Custom_NER_Spacy3.5\data\Train_spancat\dev\dev.spacy")